In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import os
from keras_retinanet.models import load_model
from keras_retinanet.utils.eval import evaluate
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image


Using TensorFlow backend.


In [2]:
model = load_model('RetinaNet_mobilenet224.h5', backbone_name='mobilenet224_1.0')
#error is safe, but can't train on this model.
#issue here: https://github.com/fizyr/keras-retinanet/issues/207

C:\Users\willg\AppData\Roaming\Python\Python36\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
validation = pd.read_csv('validation_files.csv')
validation.head()

,filename,width,height
0,coins00000109.jpg,800,533
1,lunODHd9HhRTTj8jrK79WjYvIYgK5N2k_pill.jpg,500,500
2,bottlecaps00000271.jpg,898,1300
3,ml576nXYA1vvGg0FTl7Khy3hinY2xfOf_pill.jpg,1500,1500
4,lJ6t4CTHMogiFp0DvbtUYhDlHh3cAWLH_pill.jpg,1500,1500


In [11]:
#current working directory
cwd = os.getcwd()

results = []

for idx, row in validation.iterrows():
    image_path = cwd + '\\training_images\\' + row.filename
    
    image = read_image_bgr(image_path)
    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))

    # correct for image scale
    boxes /= scale

    # each image has 300 detections, only adding the detections with strong scores
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.3:
            break
        p_row = [row.filename, row.width, row.height]
        p_row.append(label)
        p_row.extend(box)
        results.append(p_row)

In [12]:
#changing lists of lists to data frame with column names and saving as CSV
df_results = pd.DataFrame(results , columns=['filename','width','height','class','xmin','ymin','xmax','ymax'])
df_results.to_csv('results.csv', index = False, header = True)